# Explore here

In [81]:
import numpy as np, random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import os.path
import joblib
import sklearn
import sklearn.metrics as metrics
import psycopg2
import sqlite3
import requests

In [ ]:
df_rd= pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
df_rd.head(4)

In [ ]:
df_rd2= pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')
df_rd2.head(4)

In [ ]:
#df_rdMovies = df_rd.drop(['']) = pd.to_csv('../data/processed/PelisyCreditos.csv') #Dataframe con peliculas y creditos.
#df_rdPyCfilt = df_rd2.drop(['']) = pd.to_csv('../data/processed/ExtraInfo.csv')# Dataframe con Json filtrado a peticion.
conn = sqlite3.connect("../data/raw/tmdb5k.db")

df_rd.to_sql("pelis", conn, if_exists = "replace", index = False)
df_rd2.to_sql("creditos", conn, if_exists = "replace", index = False)

In [ ]:

query = """
    SELECT *
    FROM pelis
    INNER JOIN creditos
    ON pelis.title = creditos.title;
"""

df_rd3 = pd.read_sql_query(query, conn)
conn.close()

df_rd3 = df_rd3.drop_duplicates(keep='last')
df_rd3.to_csv('../data/processed/merged.csv')
df_rd3.head()
#budget	genres	homepage	id	keywords	original_language	original_title	overview	popularity	production_companies	...	runtime	spoken_languages	status	tagline	title	vote_average	vote_count	movie_id	cast	crew

In [ ]:
df_rd3.info()

In [87]:
#filtramos por cada columna con formato diccionario para crear nuevas columnas con el class name como nombre de columna y su valor como contenido de la muestra.
# Crear un nuevo DataFrame vacío para almacenar los nuevos datos
df_rd3_dicts = df_rd3.copy()[['genres', 'keywords', 'cast']]
df_rd3_raw = df_rd3.copy()[['movie_id', 'title', 'overview', 'crew']]


#### Paso 3: Transforma los datos

Como puedes ver, hay algunas columnas con formato JSON. De cada uno de los JSONs, selecciona el atributo `name` y reemplaza las columnas `genres` y `keywords`. Para la columna `cast`, selecciona los tres primeros nombres.

Las únicas columnas que quedan por modificar son `crew` (equipo) y `overview` (resumen). Para la primera columna, transfórmala para que contenga el nombre del director. Para la segunda, conviértela en una lista.

Una vez hayamos terminado de procesar las columnas y que el modelo de recomendación no se confunda, por ejemplo, entre *Jennifer Aniston* y *Jennifer Conelly*, quitaremos los espacios entre las palabras. Aplica esta función a las columnas `genres`, `cast`, `crew` y `keywords`.

Por último, reduciremos nuestro conjunto de datos combinando todas nuestras columnas convertidas anteriores en una sola columna llamada `tags` (que crearemos). Esta columna ahora tendrá todos los elementos separados por comas y luego las reemplazaremos por espacios en blanco. Debería quedar algo así:


In [88]:
import pandas as pd
import json

def sacarvalor(json_str, limit=None):
    if pd.notna(json_str):
        data = json.loads(json_str)
        return [item['name'] for item in data[:limit]] if limit else [item['name'] for item in data]
    else:
        return None
df_rd3_dicts['cast'] = df_rd3_dicts['cast'].apply(sacarvalor, limit=3)
df_rd3_dicts['genres'] = df_rd3_dicts['genres'].apply(sacarvalor)
df_rd3_dicts['keywords'] = df_rd3_dicts['keywords'].apply(sacarvalor)

In [ ]:
df_rd3_dicts.head()

In [90]:

def sacardirector(json_str):
    try:
        data = json.loads(json_str)
        for member in data:
            if member['job'] == 'Director':
                return member['name']
        return None  # 
    except (json.JSONDecodeError, TypeError):
        print(f"Error al analizar el JSON: {json_str}")
        return None

df_rd3_raw['crew'] = df_rd3_raw['crew'].apply(sacardirector)


In [ ]:
import pandas as pd

def delcolpos(df, posicion):
    df.drop(df.columns[posicion], axis=1, inplace=True)
    return df
df = delcospos(df_rd3_raw, 3)
df_rd3_raw